# Central Limit Theorem on the binomial distribution from scratch with Python

Here I am playing with the Central Limit Theorem.

I am running bernoulli trials to generate binomial distributions.

I was interested in how the results approximate the normal distribution, so I labelled their status at particular iteration rounds and plotted the result.

The code is based upon the respective example from [Data Science from Scratch](https://www.oreilly.com/library/view/data-science-from/9781492041122/).


## Libraries

In [1]:
import random
import pandas as pd
import numpy as np
import altair as alt
import math as m
from collections import Counter

## Parameters

In [2]:
p = .35
binomial_trials = 10000
total_rounds = 1000
step = 100

## Bernoulli Trials

In [3]:
def bernoulli_trial(p: float) -> int:
    return 1 if random.random() < p else 0

Let's run the trials many times.

In [4]:
trials =  []
for _ in range(binomial_trials):
    trials.append(bernoulli_trial(p))

print("First thousand trials:")
print(" ".join([str(t) for t in trials[:1000]]))
print(f"\nOnes: {sum([t == 1 for t in trials])}, Zeros: {sum([t == 0 for t in trials])}")
print("\nMean value of 'success' (ones):", np.mean(trials))

First thousand trials:
0 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0 1

In [5]:
def binomial(n: int, p: float) -> int:
    return sum(bernoulli_trial(p) for _ in range(n))

trials = binomial(binomial_trials, p)
trials

3515

## Iterating the binomial

In [6]:
def iterate_binomial(n, p, num_points):
    return [binomial(n, p) for _ in range(num_points)]

simulations = iterate_binomial(binomial_trials, p, total_rounds)

print("First hunded values:")
print(" ".join([str(results) for results in simulations[:100]]))
print("\nMean:", np.mean(simulations), "Standard deviation:", np.std(simulations))


First hunded values:
3465 3511 3479 3486 3475 3427 3558 3516 3439 3421 3494 3524 3527 3509 3465 3464 3489 3559 3452 3486 3501 3511 3571 3405 3618 3468 3397 3473 3501 3429 3519 3503 3559 3504 3500 3502 3499 3516 3505 3481 3480 3510 3548 3535 3539 3404 3498 3538 3515 3535 3397 3424 3470 3404 3475 3506 3464 3453 3548 3470 3503 3460 3516 3450 3486 3455 3598 3498 3445 3567 3437 3462 3520 3518 3446 3486 3549 3514 3590 3583 3461 3671 3463 3465 3451 3566 3458 3491 3599 3554 3470 3432 3517 3465 3621 3620 3552 3549 3482 3463

Mean: 3499.354 Standard deviation: 48.451921365411295


As we are interested in the intermediate states, we add new rounds iteratively and labeling them in the way.

In [7]:
def add_iterations(probability, trials, total_rounds, step):
    iteration_results = pd.DataFrame()

    for round in range(step, total_rounds + 1, step):
        simulations = iterate_binomial(trials, probability, round)

        counts = Counter(simulations)
        iteration_results = pd.concat(
            [
                iteration_results,
                pd.DataFrame({'count': counts.values(), 'value': counts.keys(), 'iterations': round})
            ]
        )

    return iteration_results


iteration_results = add_iterations(p, binomial_trials, total_rounds, step)
iteration_results

,count,value,iterations
0,3,3493,100
1,3,3483,100
2,1,3565,100
3,3,3445,100
4,1,3500,100
...,...,...,...
210,1,3388,1000
211,1,3619,1000
212,1,3626,1000
213,1,3603,1000


As we are interested in the total counts at each step we need to cumulatively add the value counts together over the iteration rounds.

In [8]:
def sum_counts(value_df):
    return value_df.sort_values('iterations')['count'].cumsum()

iteration_results = iteration_results.sort_values(['value', 'iterations'])

iteration_results['cumulative count'] = iteration_results.groupby('value').apply(sum_counts).values

iteration_results

,count,value,iterations,cumulative count
121,1,3342,200,1
187,1,3349,900,1
206,1,3353,800,1
74,1,3354,800,1
161,1,3356,500,1
...,...,...,...,...
160,1,3641,700,1
137,1,3647,900,1
192,1,3647,1000,2
117,1,3651,500,1


## Normal distributions

We also generate a normal distribution for reference

In [9]:
def calc_normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    return (1 + m.erf((x - mu) / m.sqrt(2) / sigma)) /2

def calc_normal_pdf(x: float, mu: float = 0, sigma: float=1) -> float:
    return m.exp(-(x-mu)**2 / (2 * sigma **2)) * 1/(m.sqrt(2 * m.pi) * sigma)

In [10]:
n = binomial_trials

mu = p * n
sigma = m.sqrt(n * p * (1 - p))

data = iteration_results.loc[iteration_results['iterations'] == iteration_results['iterations'].max(), 'value']

xs = range(min(data), max(data) + 1)
ys = [calc_normal_cdf(i + .5, mu, sigma) - calc_normal_cdf(i - .5, mu, sigma) for i in xs]

normal_dist = pd.DataFrame({'x': xs, 'y':ys})
normal_dist

,x,y
0,3364,0.000144
1,3365,0.000152
2,3366,0.000162
3,3367,0.000171
4,3368,0.000182
...,...,...
279,3643,0.000093
280,3644,0.000088
281,3645,0.000082
282,3646,0.000077


## Visualization

With the slider you can see the intermediate results at each iteration.

In [11]:
label = alt.selection_single(
    encodings=['x'], on='mouseover', nearest=True, empty='none'
)


iteration_bounds = alt.binding_range(min=iteration_results['iterations'].min(), max=iteration_results['iterations'].max(), step=step)
iteration_slider = alt.selection_single(bind=iteration_bounds, fields=['iterations'], name='Iteration', init={'iterations': iteration_results['iterations'].median()})

bar_chart = alt.Chart(iteration_results).mark_bar().encode(
    alt.X('value:Q', scale=alt.Scale(domain=(iteration_results['value'].min(), iteration_results['value'].max()))),
    alt.Y('cumulative count:Q', scale=alt.Scale(domain=(iteration_results['cumulative count'].min(), iteration_results['cumulative count'].max()))),
    opacity=alt.value(0.75)
)


line_chart = alt.Chart(normal_dist).mark_line().encode(
    alt.X('x'), alt.Y('y',),color=alt.value('darkred')
)

bar_chart = alt.layer(
    bar_chart.add_selection(iteration_slider).transform_filter(iteration_slider),
    bar_chart.mark_circle().encode(opacity=alt.condition(label, alt.value(1), alt.value(0))).add_selection(label).transform_filter(iteration_slider),
    bar_chart.mark_text(dx=5, dy=-5, align='left', strokeWidth=.5, stroke='black').encode(text=alt.Text('cumulative count:Q')).transform_filter(label).transform_filter(iteration_slider)
)

alt.layer(
    bar_chart,
    line_chart,    
).resolve_scale(y='independent').properties(width=800)

alt.LayerChart(...)